# Usando Picat
> Usando Jupyter Notebook com Picat em um post

- toc: true
- author: Marcio Minicz
- categories: [picat, jupyter]

In [5]:
#hide
import os
!wget -c http://picat-lang.org/download/picat30_3_linux64.tar.gz
!tar -xf picat30_3_linux64.tar.gz
os.environ['PATH'] = os.environ['PATH'] + ':/content/Picat'
!pip install ipicat > /dev/null

--2020-12-27 12:34:41--  http://picat-lang.org/download/picat30_3_linux64.tar.gz
Resolving picat-lang.org (picat-lang.org)... 64.71.35.59
Connecting to picat-lang.org (picat-lang.org)|64.71.35.59|:80... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



# Loding Picat Extension

In [6]:
%load_ext ipicat

The ipicat extension is already loaded. To reload it, use:
  %reload_ext ipicat


# Executing a small program

In [7]:
%%picat

main =>
  X is mod( -( 2 * 3**4), 5),
  Y is mod(  (-2 * 3**4), 5),
  println(X),
  println(Y).

3
-2



In [8]:
%%picat

/*

  Perfect square placement in Picat.

  See CSPLib for description of the problem:
  http://www.cs.st-andrews.ac.uk/~ianm/CSPLib/prob/prob009/index.html

  """
  The perfect square placement problem (also called the squared square problem) 
  is to pack a set of squares with given integer sizes into a bigger square in 
  such a way that no squares overlap each other and all square borders are 
  parallel to the border of the big square. For a perfect placement problem, 
  all squares have different sizes. The sum of the square surfaces is equal 
  to the surface of the packing square, so that there is no spare capacity. 
  A simple perfect square placement problem is a perfect square placement 
  problem in which no subset of the squares (greater than one) are placed 
  in a rectangle. 
  """

  The problems files from CSPLib
  http://www.cs.st-andrews.ac.uk/~ianm/CSPLib/prob/prob009/results
  are places here
  http://www.hakank.org/minizinc/perfect_square_placement/


  Note: this model use just diffn/4 and cumulative/4 and don't 
  (necessary) conform to the constraint given above that
    "all square borders are parallel to the border of the big square"
  

  Model created by Hakan Kjellerstrand, hakank@gmail.com
  See also my Picat page: http://www.hakank.org/picat/

*/

import cp.

main => go.

go ?=>
   time2(perfect_square(ffd, split)),
   % time2(perfect_square(ffc, split)),
   % fail,
   nl.

go => true.

go2 =>
   selection(VariableSelection),
   choice(ValueSelection),
   Timeout = 10000,
   Success = [],
   Fails = [],
   foreach(Var in VariableSelection, Val in ValueSelection) 
       writeln([Var,Val]),
       time_out(time2($perfect_square(Var, Val)), Timeout, Result),
       writeln(Result),
       if Result == success then
          Success := Success ++ [[Var,Val]]
       end,
       if Result == time_out then
          Fails := Fails ++ [[Var, Val]]
       end
   end,
   writeln(success=Success),
   writeln(fails=Fails),
   nl.

perfect_square(Var, Val) =>
   % N = 6,
   % A =   [1,1,1,1,1,2,3,3,3],


   % Problem from 
   % http://www.maa.org/editorial/mathgames/mathgames_12_01_03.html
   N = 14, % size of main square
   A = [1,1,1,1,2,3,3,3,5,6,6,8], % Sizes

   % Problem 1 from 
   % http://www.cs.st-andrews.ac.uk/~ianm/CSPLib/prob/prob009/results
   % N = 112, % size of main square
   % A = [2,4,6,7,8,9,11,15,16,17,18,19,24,25,27,29,33,35,37,42,50],

   % From Mathematica Tech Conference 2018:
   % http://www.wolfram.com/broadcast/video.php?c=452&p=3&v=2434
   % Nope, no solution
   % N = 36+32,  %size of main square
   % squares and double squares
   % A = [1,2,4,8,9,16,18,26,32,36,49],

   Size = length(A),

   X = new_list(Size),
   X :: 1..N,

   Y = new_list(Size),
   Y :: 1..N,

   % constraints

   cumulative(X,A,A,N),
   cumulative(Y,A,A,N),

   diffn([X ++ A, Y ++ A]), 
   % diffn_me(X,Y,A,A),
   % diffn_nonstrict(X,Y,A,A),     

   foreach(I in 1..Size)
     X[I] + A[I] #=< N+1,
     Y[I] + A[I] #=< N+1
   end,

   % println(x=X),
   % println(y=Y),

   Vars = X ++ Y,
   println(solve),
   solve([Var, Val], Vars),

   println(a=A),
   println(x=X),
   println(y=Y),

   nl.


% Variable selection
selection(Variable) => 
  Variable = [backward,constr,degree,ff,ffc,forward,inout,leftmost,max,min,rand_var].

% Value selection
choice(Value) => 
  Value = [down,updown,split,reverse_split,rand_val,up].

/*
From MiniZinc's diffn_nonstrict/4
*/

diffn_nonstrict(X,Y,DX,DY) =>
  N = X.len,
  foreach(I in 1..N, J in I+1..N)
    DX[I] #= 0 #\/
    DX[J] #= 0 #\/
    DY[I] #= 0 #\/
    DY[J] #= 0 #\/
    X[I] + DX[I] #=< X[J] #\/
    Y[I] + DY[I] #=< Y[J] #\/
    X[J] + DX[J] #=< X[I] #\/
    Y[J] + DY[J] #=< Y[I]
  end.

/*
From MiniZinc's diffn/4
*/
diffn_me(X,Y,DX,DY) =>
  N = X.len,
  foreach(I in 1..N, J in I+1..N) 
    X[I] + DX[I] #<= X[J] #\/
    Y[I] + DY[I] #<= Y[J]  #\/
    X[J] + DX[J] #<= X[I] #\/
    Y[J] + DY[J] #<= Y[I]
  end.

solve
a = [1,1,1,1,2,3,3,3,5,6,6,8]
x = [14,8,14,7,12,12,12,9,7,9,1,1]
y = [14,14,8,7,12,12,12,9,7,9,1,1]


CPU time 0.024 seconds. Backtracks: 0



